In [2]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px

In [3]:
baseline = Microsimulation()

In [7]:
reform = Reform.from_dict(
    {
        "gov.contrib.congress.wyden_smith.actc_lookback": {
            "2024-01-01.2025-12-31": True
        },
        "gov.contrib.congress.wyden_smith.per_child_actc_phase_in": {
            "2023-01-01.2025-12-31": True
        },
        "gov.irs.credits.ctc.amount.base[0].amount": {"2024-01-01.2025-12-31": 2100},
        "gov.irs.credits.ctc.refundable.individual_max": {
            "2023-01-01.2023-12-31": 1800,
            "2024-01-01.2024-12-31": 1900,
            "2025-01-01.2025-12-31": 2100,
        },
    },
    country_id="us",
)
reformed = Microsimulation(reform=reform)

In [39]:
# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

In [40]:
# Calculate for each year
for year in range(2023, 2026):
    baseline_household = baseline.calculate(
        "household_net_income", period=year, map_to="household"
    )
    reformed_household = reformed.calculate(
        "household_net_income", period=year, map_to="household"
    )

    # Calculate the ctc_qualifying_children at the household level
    ctc_qualifying_children = baseline.calculate(
        "ctc_qualifying_children", period=year, map_to="household"
    )

    # Get state codes at the household level
    state_codes = baseline.calculate("state_code", period=year, map_to="household")

    # Calculate the baseline weight at the household level
    baseline_weight = baseline_household.weights

    # Calculate adjusted gross income
    agi = baseline.calculate("adjusted_gross_income", period=year, map_to="household")

    # Calculate the difference for all households
    difference = reformed_household - baseline_household

    # Create a DataFrame with all the results
    df_year = pd.DataFrame(
        {
            "Year": year,
            "State": state_codes,
            "Difference": difference,
            "CTC_Qualifying_Children": ctc_qualifying_children,
            "Weight": baseline_weight,
            "AGI": agi,
        }
    )

    # Create a new column for weighted difference
    df_year["Weighted_Difference"] = df_year["Difference"] * df_year["Weight"]

    # Filter the DataFrame to households where ctc_qualifying_children >= 1 and AGI between 2,500 and 50,000
    df_filtered = df_year[
        (df_year["CTC_Qualifying_Children"] >= 1)
        & (df_year["AGI"] >= 2500)
        & (df_year["AGI"] <= 50000)
    ]

    # Group by State and sum the Weighted_Difference and Weight
    df_grouped = df_filtered.groupby("State")[["Weighted_Difference", "Weight"]].sum()

    # Calculate the total Difference separately
    total_difference = df_filtered.groupby("State")["Difference"].sum()

    # Add the total Difference to df_grouped
    df_grouped["Total_Difference"] = total_difference

    # Calculate the weighted average difference
    df_grouped["Mean_Difference"] = (
        df_grouped["Weighted_Difference"] / df_grouped["Weight"]
    )

    # Reset the index to make State a column again
    df_grouped = df_grouped.reset_index()

    # Add the Year column
    df_grouped["Year"] = year

    # Select and rename columns
    df_grouped = df_grouped[
        ["Year", "State", "Mean_Difference", "Weighted_Difference", "Weight"]
    ]
    df_grouped = df_grouped.rename(columns={"Weight": "Total_Weight"})

    # Append to the all_results DataFrame
    all_results = pd.concat([all_results, df_grouped], ignore_index=True)

# Sort the DataFrame by State and Year
all_results = all_results.sort_values(["State", "Year"])

# Reset the index
all_results = all_results.reset_index(drop=True)

# Remove any remaining rows with NaN values
all_results = all_results.dropna()

# Display the results
print(all_results)

     Year State  Mean_Difference  Weighted_Difference   Total_Weight
0    2023    AK       963.892457         1.145752e+07   11886.719986
1    2024    AK      1071.446652         1.233967e+07   11516.829971
2    2025    AK      1209.229954         1.355280e+07   11207.789963
3    2023    AL       494.327029         6.973749e+07  141075.609558
4    2024    AL       607.928490         7.953142e+07  130823.649780
..    ...   ...              ...                  ...            ...
148  2024    WV       476.600689         2.296101e+07   48176.619934
149  2025    WV       549.055855         2.447819e+07   44582.329987
150  2023    WY       323.264818         3.918054e+06   12120.260025
151  2024    WY       401.925466         4.400654e+06   10948.930023
152  2025    WY       459.486492         5.030885e+06   10948.930023

[153 rows x 5 columns]


In [43]:
# Remove any remaining rows with NaN values
all_results = all_results.dropna()

# Display the results
print(all_results)

# Export the results to a CSV file
csv_filename = "all_results.csv"
all_results.to_csv(csv_filename, index=False)

print(f"Results have been exported to '{csv_filename}'")

     Year State  Mean_Difference  Weighted_Difference   Total_Weight
0    2023    AK       963.892457         1.145752e+07   11886.719986
1    2024    AK      1071.446652         1.233967e+07   11516.829971
2    2025    AK      1209.229954         1.355280e+07   11207.789963
3    2023    AL       494.327029         6.973749e+07  141075.609558
4    2024    AL       607.928490         7.953142e+07  130823.649780
..    ...   ...              ...                  ...            ...
148  2024    WV       476.600689         2.296101e+07   48176.619934
149  2025    WV       549.055855         2.447819e+07   44582.329987
150  2023    WY       323.264818         3.918054e+06   12120.260025
151  2024    WY       401.925466         4.400654e+06   10948.930023
152  2025    WY       459.486492         5.030885e+06   10948.930023

[153 rows x 5 columns]
Results have been exported to 'all_results.csv'
